# Upper Triangular Matrix Vector Multiply Routines

This notebook walks you through how to implement $ y := U x + y $ where $ U $ is upper triangular.

## Getting started

We will use some functions that are part of our laff library (of which this function will become a part) as well as some routines from the FLAME API (Application Programming Interface) that allow us to write code that closely resembles how we typeset algorithms using the FLAME notation.  These functions are imported with the "import laff as laff" and "import flame" statements.

## Algorithm that takes dot products

<img src="https://studio.edx.org/c4x/UTAustinX/UT.5.01x/asset/432_Trmvp_un_unb_var1.png" alt="Alternative matrix-vector multiplication via dot products algorithm" width="50%">

## The routine <br> <code> Tmvmult_un_unb_var1( U, x, y ) </code>

This routine, given upper triangular $ U \in \mathbb{R}^{n \times n} $, $ x \in \mathbb{R}^n $, and $ y \in \mathbb{R}^n $, computes $ y := U x + y $.  The "_un_" in the name of the routine indicates this is the "upper, no transpose" matrix-vector multiplication.  

The specific laff functions we will use are 
<ul>
<li> <code> laff.dots( x, y, alpha ) </code> which computes $ \alpha := x^T y + \alpha $.  </li>
</ul>

Use the <a href="https://studio.edx.org/c4x/UTAustinX/UT.5.01x/asset/index.html"> Spark webpage</a> to generate a code skeleton.  (Make sure you adjust the name of the routine.)

In [1]:
import flame
import laff as laff

def Tmvmult_un_unb_var1(U, x, y):

    UTL, UTR, \
    UBL, UBR  = flame.part_2x2(U, \
                               0, 0, 'TL')

    xT, \
    xB  = flame.part_2x1(x, \
                         0, 'TOP')

    yT, \
    yB  = flame.part_2x1(y, \
                         0, 'TOP')

    while UTL.shape[0] < U.shape[0]:

        U00,  u01,       U02,  \
        u10t, upsilon11, u12t, \
        U20,  u21,       U22   = flame.repart_2x2_to_3x3(UTL, UTR, \
                                                         UBL, UBR, \
                                                         1, 1, 'BR')

        x0,   \
        chi1, \
        x2    = flame.repart_2x1_to_3x1(xT, \
                                        xB, \
                                        1, 'BOTTOM')

        y0,   \
        psi1, \
        y2    = flame.repart_2x1_to_3x1(yT, \
                                        yB, \
                                        1, 'BOTTOM')

        #------------------------------------------------------------#

        laff.dots(upsilon11, chi1, psi1)
        laff.dots(u12t, x2, psi1)

        #------------------------------------------------------------#

        UTL, UTR, \
        UBL, UBR  = flame.cont_with_3x3_to_2x2(U00,  u01,       U02,  \
                                               u10t, upsilon11, u12t, \
                                               U20,  u21,       U22,  \
                                               'TL')

        xT, \
        xB  = flame.cont_with_3x1_to_2x1(x0,   \
                                         chi1, \
                                         x2,   \
                                         'TOP')

        yT, \
        yB  = flame.cont_with_3x1_to_2x1(y0,   \
                                         psi1, \
                                         y2,   \
                                         'TOP')

    flame.merge_2x1(yT, \
                    yB, y)

## Testing

Let's quickly test the routine by creating a 4 x 4 matrix and related vectors, performing the computation.

In [2]:
from numpy import random
from numpy import matrix

U = matrix( random.rand( 4,4 ) )
x = matrix( random.rand( 4,1 ) )
y = matrix( random.rand( 4,1 ) )
yold = matrix( random.rand( 4,1 ) )

# Notice that U is not upper triangular.  We will only use the upper triangular part.

print( 'U before =' )
print( U )

print( 'x before =' )
print( x )

print( 'y before =' )
print( y )

U before =
[[0.04961322 0.946872   0.59623435 0.37675078]
 [0.68937096 0.91151027 0.07253809 0.52489933]
 [0.51931519 0.99264289 0.67808045 0.17418648]
 [0.68982321 0.36913353 0.29078887 0.55231132]]
x before =
[[0.73162926]
 [0.4265258 ]
 [0.30704265]
 [0.53143201]]
y before =
[[0.82370409]
 [0.51850355]
 [0.36135024]
 [0.87765447]]


In [3]:
import numpy as np
laff.copy( y, yold )   # save the original vector y

Tmvmult_un_unb_var1( U, x, y )

print( 'y after =' )
print( y )

print( 'y - ( np.triu( U ) * x + yold ) = ' ) #np.triu makes the matrix upper triangular
print( y - ( np.triu( U ) * x + yold ) )

y after =
[[1.6471547 ]
 [1.20850679]
 [0.66211813]
 [1.17117038]]
y - ( np.triu( U ) * x + yold ) = 
[[-2.22044605e-16]
 [ 0.00000000e+00]
 [ 1.11022302e-16]
 [ 0.00000000e+00]]


Bingo, it seems to work!  (Notice that we are doing floating point computations, which means that due to rounding you may not get an exact "0".)

## Watch your code in action!

Copy and paste the code into <a href="http://edx-org-utaustinx.s3.amazonaws.com/UT501x/PictureFlame/PictureFLAME.html"> PictureFLAME </a>, a webpage where you can watch your routine in action.  Just cut and paste into the box.  

Disclaimer: we implemented a VERY simple interpreter.  If you do something wrong, we cannot guarantee the results.  But if you do it right, you are in for a treat.

If you want to reset the problem, just click in the box into which you pasted the code and hit "next" again.

## Algorithm that uses axpys

<img src="https://studio.edx.org/c4x/UTAustinX/UT.5.01x/asset/432_Trmvp_un_unb_var2.png" alt="Alternative matrix-vector multiplication via dot products algorithm" width="50%">

## The routine <br> <code> Tmvmult_un_unb_var2( U, x, y ) </code>

This routine, given upper triangular $ U \in \mathbb{R}^{n \times n} $, $ x \in \mathbb{R}^n $, and $ y \in \mathbb{R}^n $, computes $ y := U x + y $.  The "_un_" in the name of the routine indicates this is the "upper triangular, no transpose" matrix-vector multiplication.  

The specific laff functions we will use are 
<ul>
<li> <code> laff.axpy( alpha, x, y ) </code> which computes $ y := \alpha x +  y  $.  </li>
</ul>

Use the <a href="https://studio.edx.org/c4x/UTAustinX/UT.5.01x/asset/index.html"> Spark webpage</a> to generate a code skeleton.  (Make sure you adjust the name of the routine.)

In [4]:
def Tmvmult_un_unb_var2(U, x, y):

    UTL, UTR, \
    UBL, UBR  = flame.part_2x2(U, \
                               0, 0, 'TL')

    xT, \
    xB  = flame.part_2x1(x, \
                         0, 'TOP')

    yT, \
    yB  = flame.part_2x1(y, \
                         0, 'TOP')

    while UTL.shape[0] < U.shape[0]:

        U00,  u01,       U02,  \
        u10t, upsilon11, u12t, \
        U20,  u21,       U22   = flame.repart_2x2_to_3x3(UTL, UTR, \
                                                         UBL, UBR, \
                                                         1, 1, 'BR')

        x0,   \
        chi1, \
        x2    = flame.repart_2x1_to_3x1(xT, \
                                        xB, \
                                        1, 'BOTTOM')

        y0,   \
        psi1, \
        y2    = flame.repart_2x1_to_3x1(yT, \
                                        yB, \
                                        1, 'BOTTOM')

        #------------------------------------------------------------#

        laff.axpy(chi1, u01, y0)
        laff.axpy(chi1, upsilon11, psi1)

        #------------------------------------------------------------#

        UTL, UTR, \
        UBL, UBR  = flame.cont_with_3x3_to_2x2(U00,  u01,       U02,  \
                                               u10t, upsilon11, u12t, \
                                               U20,  u21,       U22,  \
                                               'TL')

        xT, \
        xB  = flame.cont_with_3x1_to_2x1(x0,   \
                                         chi1, \
                                         x2,   \
                                         'TOP')

        yT, \
        yB  = flame.cont_with_3x1_to_2x1(y0,   \
                                         psi1, \
                                         y2,   \
                                         'TOP')

    flame.merge_2x1(yT, \
                    yB, y)

## Testing

Let's quickly test the routine by creating a 4 x 4 matrix and related vectors, performing the computation.

In [5]:
from numpy import random
from numpy import matrix

U = matrix( random.rand( 4,4 ) )
x = matrix( random.rand( 4,1 ) )
y = matrix( random.rand( 4,1 ) )
yold = matrix( random.rand( 4,1 ) )

# U is not upper triangular.  We will only use the upper triangular part.

print( 'U before =' )
print( U )

print( 'x before =' )
print( x )

print( 'y before =' )
print( y )

U before =
[[0.23054024 0.99024422 0.33290809 0.34781555]
 [0.00327732 0.45465462 0.612628   0.10948577]
 [0.11185153 0.70161225 0.83398733 0.72502455]
 [0.78833859 0.17530866 0.67385587 0.79269805]]
x before =
[[0.97900952]
 [0.37831969]
 [0.83605927]
 [0.99673974]]
y before =
[[0.55192227]
 [0.41371952]
 [0.88129193]
 [0.64746067]]


In [6]:
laff.copy( y, yold )   # save the original vector y

Tmvmult_un_unb_var2( U, x, y )

print( 'y after =' )
print( y )

print( 'y - ( np.triu( U ) * x + yold ) = ' ) #np.triu makes the matrix upper triangular
print( y - ( np.triu( U ) * x + yold ) )

y after =
[[1.77726473]
 [1.20704645]
 [2.30121554]
 [1.43757432]]
y - ( np.triu( U ) * x + yold ) = 
[[0.]
 [0.]
 [0.]
 [0.]]


Bingo, it seems to work!  (Notice that we are doing floating point computations, which means that due to rounding you may not get an exact "0".)

## Watch your code in action!

Copy and paste the code into <a href="http://edx-org-utaustinx.s3.amazonaws.com/UT501x/PictureFlame/PictureFLAME.html"> PictureFLAME </a>, a webpage where you can watch your routine in action.  Just cut and paste into the box.  

Disclaimer: we implemented a VERY simple interpreter.  If you do something wrong, we cannot guarantee the results.  But if you do it right, you are in for a treat.

If you want to reset the problem, just click in the box into which you pasted the code and hit "next" again.